In [1]:
import sys
sys.path.append('../pycore/')
from utils import extract_details_from_rxnid

import pandas as pd

In [2]:
df_prot = pd.read_excel('../scRBA_build_GAMS_model/input/PROTEIN_stoich_curation.xlsx')
prot_mw = dict()
for i in df_prot.index:
    prot_mw[df_prot.id[i]] = float(df_prot.loc[i, 'MW (g/mmol)'])
prot_mw['PROTDUMMY'] = 44.026303

In [3]:
df_enz = pd.read_excel('../scRBA_build_GAMS_model/input/ENZYME_stoich_curation_2021-10-01.xlsx')
enz_mw = dict()
for i in df_enz.index:
    enzload_id = df_enz.id[i].replace('RXN-', 'ENZLOAD-')
    enz_mw[enzload_id] = float(df_enz.loc[i, 'MW (g/mmol)'])

In [4]:
class RBA_result:
    def __init__(self, growth_rate='', raw_flux='', metabolic_flux='',
                 ribo_capacity_usage=0, proteome_capacity_usage=0,
                 proteome_allocation='', proteome_allocation_by_rxns='',
                 protein_mw='', enzyme_mw=''):
        self.growth_rate = growth_rate
        self.raw_flux = raw_flux
        self.metabolic_flux = metabolic_flux
        self.ribo_capacity_usage = ribo_capacity_usage
        self.proteome_capacity_usage = proteome_capacity_usage
        self.proteome_allocation = proteome_allocation
        self.proteome_allocation_by_rxns = proteome_allocation_by_rxns
        self.protein_mw = protein_mw
        self.enzyme_mw = enzyme_mw
        
    def load_raw_flux(self, filepath='./runRBA.flux.txt'):
        with open(filepath) as f:
            text = f.read().split('\n')
        text = [i for i in text if i != '']
        
        fluxdict = dict()
        for i in text:
            r,_,v = i.split('\t')
            fluxdict[r] = float(v)
        self.raw_flux = fluxdict
        self.growth_rate = fluxdict['BIOSYN-BIODIL']
        
    def calculate_metabolic_flux(self):
        metfluxdict = dict()
        for k,v in self.raw_flux.items():
            if k[:4] == 'RXN-':
                _,rxn,rxn_dir,_ = extract_details_from_rxnid(k)
                if rxn_dir == 'FWD':
                    rval = v
                elif rxn_dir == 'REV':
                    rval = -v
                
                if rxn not in metfluxdict.keys():
                    metfluxdict[rxn] = rval
                else:
                    metfluxdict[rxn] += rval
        self.metabolic_flux = metfluxdict
        
    def calculate_ribo_capacity_usage(self):
        rrna_cap = 0.8 * self.raw_flux['BIOSYN-RNA']
        rrna_unused = self.raw_flux['BIOSYN-RNA7']
        self.ribo_capacity_usage = (rrna_cap - rrna_unused) / rrna_cap
        
    def calculate_proteome_capacity_usage(self):
        if 'BIOSYN-PROTDUMMY2' in self.raw_flux.keys():
            dummyload = self.raw_flux['BIOSYN-PROTDUMMY2']
        else:
            dummyload = 0
        pload = self.raw_flux['BIOSYN-PROTMODELED']
        procap = self.raw_flux['BIOSYN-PROTTOBIO']
        self.proteome_capacity_usage = (pload - dummyload) / procap
        
    def calculate_proteome_allocation(self):
        if self.protein_mw == '':
            print('Cannot calculate. Need to load in protein molecular weight data ' + \
                  'as self.protein_mw = dictionary_of_protein_mw')
            return None
            
        protfluxdict = dict()
        for k,v in self.raw_flux.items():
            if k[:7] == 'PROSYN-':
                prot_id = k[7:]
                mw = self.protein_mw[prot_id]
                if '_' in prot_id:
                    prot_id = prot_id.split('_')[0]
                if prot_id not in protfluxdict.keys():
                    protfluxdict[prot_id] = v*mw
                else:
                    protfluxdict[prot_id] += v*mw
        ptot = float(self.raw_flux['BIOSYN-PROTTOBIO'])
        protfluxdict = {k:v/ptot for k,v in protfluxdict.items()}
        self.proteome_allocation = protfluxdict
        
    def calculate_proteome_allocation_by_rxns(self):
        if self.enzyme_mw == '':
            print('Cannot calculate. Need to load in enzyme molecular weight data ' + \
                  'as self.enzyme_mw = dictionary_of_enzyme_mw')
            return None
        
        enzreq = dict()
        for k,v in self.raw_flux.items():
            if k[:8] == 'ENZLOAD-' and len(k.split('-')):
                mw = self.enzyme_mw[k]
                _,rxn,_,_ = extract_details_from_rxnid(k)
                if rxn not in enzreq.keys():
                    enzreq[rxn] = v*mw
                else:
                    enzreq[rxn] += v*mw
        ptot = float(self.raw_flux['BIOSYN-PROTTOBIO'])
        enzreq = {k:v/ptot for k,v in enzreq.items()}
        self.proteome_allocation_by_rxns = enzreq
        
    def calculate_all(self):
        self.calculate_metabolic_flux()
        self.calculate_ribo_capacity_usage()
        self.calculate_proteome_capacity_usage()
        self.calculate_proteome_allocation()
        self.calculate_proteome_allocation_by_rxns()
        
    def load_and_calculate(self, filepath='./runRBA.flux.txt'):
        self.load_raw_flux(filepath)
        self.calculate_all()
    
    def save_to_json(self, filepath):
        import json
        with open(filepath, 'w') as f:
            json.dump(self.__dict__, f)
            
    def load_from_json(self, filepath):
        import json
        with open(filepath) as f:
            resdict = json.load(f)
        for k,v in resdict.items():
            self.__setattr__(k,v)

In [5]:
x = RBA_result(protein_mw=prot_mw, enzyme_mw=enz_mw)
x.load_and_calculate(filepath='./runRBA.flux.txt')

In [6]:
x.proteome_capacity_usage / 0.55

0.9705584417184578

In [7]:
x.ribo_capacity_usage

0.6220388464898756

In [8]:
x.save_to_json('./RBA_result.json')

In [9]:
y = RBA_result()

In [10]:
y.load_from_json('./RBA_result.json')

In [14]:
y.proteome_capacity_usage / 0.55

0.9705584417184578

In [15]:
y.ribo_capacity_usage

0.6220388464898756

In [ ]:
y

In [6]:
x.load_raw_flux('./runRBA.flux.txt')

In [7]:
x.enzyme_mw = enz_mw
x.calculate_proteome_allocation_by_rxns()

In [8]:
sum(x.proteome_allocation_by_rxns.values())

0.4709097887012757

In [9]:
x.calculate_ribo_capacity_usage()

In [11]:
x.ribo_capacity_usage

0.6220388464898756

In [16]:
x.calculate_metabolic_flux()

In [17]:
x.metabolic_flux

{'EX_pnto__R_e': -3.40122e-05,
 'EX_4abz_e': -1.126441e-05,
 'EX_nh4_e': -1.11900495351,
 'EX_co2_e': 15.99595081655,
 'EX_glc__D_e': -8.65473097791,
 'EX_etoh_e': 14.46532858071,
 'EX_gcald_e': 1.126441e-05,
 'EX_h_e': 0.2988003677,
 'EX_fe2_e': -0.00013298497,
 'EX_inost_e': -0.00230023119,
 'EX_nac_e': -0.00108527651,
 'EX_o2_e': -1.15888512763,
 'EX_pi_e': -0.0711382606,
 'EX_k_e': -0.10888660756,
 'EX_pydxn_e': -1.649454e-05,
 'EX_ribflv_e': -0.00019854466,
 'EX_so4_e': -0.05589673822,
 'EX_thm_e': -1.447561e-05,
 'EX_h2o_e': 4.7329575101,
 'EX_cu2_e': -2.073428e-05,
 'EX_mn2_e': -2.064715e-05,
 'EX_zn2_e': -0.00068924658,
 'EX_mg2_e': -0.01156925661,
 'EX_ca2_e': -0.00023191966,
 'EX_cobalt2_e': -1.855e-08,
 'EX_ni2_e': -8.04533e-06,
 '13GS_c': 0.16688861366,
 '16GS_c': 0.04391817021,
 'PRMICI_c': 0.03425496699,
 'AATA_c': 0.03489272417,
 'IPPMIb_c': -0.04836497043,
 'HCITR_m': 0.03489272417,
 'OMCDC_c': 0.04836497043,
 'BPNT_c': 0.05027542721,
 'DHQTi_c': 0.0621848821,
 'DHQS_c'